In [13]:
import os
import cv2
from PIL import Image

---

In [14]:
path_data = "D:\\study\\machine_learning\\classification_fish\\data\\Fish_Dataset"
output_folder = "D:\\study\\machine_learning\\classification_fish\\clustering\\data\\Fish_Dataset"

In [15]:
# Define the folder names
folder_names = ['Black Sea Sprat', 'Gilt Head Bream', 'Hourse Mackerel', 'Red Mullet',
               'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']

# Create the folders if they don't exist
for folder_name in folder_names:
    folder_path = os.path.join(output_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)

In [16]:
folders = os.listdir(path_data)
for folder in folders:
    folder_path = os.path.join(path_data, folder, folder)
    k = 1
    for i in range (1, 1001, 20):
        img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")
        img_save = os.path.join(output_folder, folder, f"{str(k).zfill(5)}.png")
        img = Image.open(img_path)
        resized_img = img.resize((128, 128), resample=Image.BICUBIC) # thay đổi giá trị của new_width, new_height và resample tùy ý
        resized_img.save(img_save)
        k = k + 1

In [17]:
path_data = "D:\\study\\machine_learning\\classification_fish\\data\\NA_Fish_Dataset"

In [18]:
folders = os.listdir(path_data)
for folder in folders:
    folder_path = os.path.join(path_data, folder)
    k = 51
    # if folder == "Trout":
    #     for i in range (1, 31):
    #         img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")
    #         img_save = os.path.join(output_folder, folder, f"{str(k).zfill(5)}.png")
    #         img = Image.open(img_path)
    #         resized_img = img.resize((128, 128), resample=Image.BICUBIC) # thay đổi giá trị của new_width, new_height và resample tùy ý
    #         resized_img.save(img_save)
    #         k = k + 1
    # else:
    for i in range (1, 51):
        img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")
        img_save = os.path.join(output_folder, folder, f"{str(k).zfill(5)}.png")
        img = Image.open(img_path)
        resized_img = img.resize((128, 128), resample=Image.BICUBIC) # thay đổi giá trị của new_width, new_height và resample tùy ý
        resized_img.save(img_save)
        k = k + 1

---

In [8]:
from tensorflow.keras.models import Model, load_model
import cv2
import numpy as np
import tensorflow as tf

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

In [4]:
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = label(y_true_in > 0.5)
    y_pred = label(y_pred_in > 0.5)
    
    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.array(np.mean(metric), dtype=np.float32)

def my_iou_metric(label, pred):
    metric_value = tf.py_function(iou_metric_batch, [label, pred], tf.float32)
    return metric_value

In [20]:
path = "D:\\study\\machine_learning\\classification_fish\\clustering\\data\\Fish_Dataset"
images = [] # mảng chứa dữ liệu
folders = os.listdir(path)
for folder in folders:
    folder_path = os.path.join(path, folder)
    for i in range(1, 101):
        img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")
        # print(img_path)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))

        images.append(img)

In [21]:
images = np.array(images)

In [23]:
images.shape

(900, 128, 128, 3)

In [24]:
import os

output_folder = 'D:\\study\\machine_learning\\classification_fish\\clustering\\data\\Fish_Dataset_GT'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [2]:
# Define the folder names
folder_names = ['Black Sea Sprat', 'Gilt Head Bream', 'Hourse Mackerel', 'Red Mullet',
               'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']

# Create the folders if they don't exist
for folder_name in folder_names:
    folder_path = os.path.join(output_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)

In [5]:
model = load_model('D:\\study\\machine_learning\\classification_fish\\model\\unet_1.hdf5', 
                   custom_objects={'my_iou_metric': my_iou_metric})

In [26]:
img = model.predict(images, verbose=1)

29/29 [==============================] - 17s 603ms/step


In [27]:
img = (img > 0.5).astype(np.uint8)

In [28]:
img_gt = np.squeeze(img)

In [29]:
# Iterate through the images
for i in range(len(img_gt)):
    # Get the folder name and image index
    folder_index = i // 100
    folder_name = folder_names[folder_index]
    image_index = i % 100

    # Format the image index as a 5-digit string
    image_index_str = str(image_index + 1).zfill(5)

    # Create the file name
    filename = f'{image_index_str}.png'

    # Convert the predicted array to a binary image
    img_binary = (img_gt[i] > 0.5).astype(np.uint8) * 255

    # Create the image object from the binary array
    pred_image = Image.fromarray(img_binary, mode='L')

    # Save the image to the corresponding folder
    folder_path = os.path.join(output_folder, folder_name)
    output_path = os.path.join(folder_path, filename)
    pred_image.save(output_path)

---

In [30]:
path_img = "D:\\study\\machine_learning\\classification_fish\\clustering\\data\\Fish_Dataset"
path_gt = "D:\\study\\machine_learning\\classification_fish\\clustering\\data\\Fish_Dataset_GT"
output_folder = "D:\\study\\machine_learning\\classification_fish\\clustering\\data\\Fish_Dataset_Segment"

In [31]:
# Define the folder names
folder_names = ['Black Sea Sprat', 'Gilt Head Bream', 'Hourse Mackerel', 'Red Mullet',
               'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']

# Create the folders if they don't exist
for folder_name in folder_names:
    folder_path = os.path.join(output_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)

In [37]:
folders = os.listdir(path_img)
for folder in folders:
    folder_path = os.path.join(path_img, folder)
    folder_gt = os.path.join(path_gt, folder)
    print(folder_path)
    print(folder_gt)
    for i in range(1, 101):
        img_path = os.path.join(folder_path, f"{str(i).zfill(5)}.png")
        mask_path = os.path.join(folder_gt, f"{str(i).zfill(5)}.png")
        # đọc ảnh
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))
        mask = cv2.imread(mask_path)
        mask = cv2.resize(mask, (128, 128))

        # Chuyển ảnh mask sang ảnh binary
        gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)

        # Tìm contours trên ảnh binary
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # Lặp qua từng contour
        for contour in contours:
            # Tìm bounding box của contour
            x,y,w,h = cv2.boundingRect(contour)
            
            # Kiểm tra kích thước bounding box
            if w > 25 and h > 25:
                # Kiểm tra màu của pixel tương ứng trên ảnh ground truth
                if mask[y:y+h, x:x+w].max() == 255:
                    # Lấy phần ảnh tương ứng từ ảnh gốc
                    obj = cv2.bitwise_and(img[y:y+h, x:x+w], mask[y:y+h, x:x+w])
                    # Lưu ảnh vào thư mục D:\study\machine_learning\classification_fish\data\NA_Fish_Dataset\ folder
                    cv2.imwrite(f"D:\\study\\machine_learning\\classification_fish\\clustering\\data\\Fish_Dataset_Segment\\{folder}\\{str(i).zfill(5)}.png", obj)

D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset\Black Sea Sprat
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset_GT\Black Sea Sprat
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset\Gilt Head Bream
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset_GT\Gilt Head Bream
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset\Hourse Mackerel
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset_GT\Hourse Mackerel
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset\Red Mullet
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset_GT\Red Mullet
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset\Red Sea Bream
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset_GT\Red Sea Bream
D:\study\machine_learning\classification_fish\clustering\data\Fish_Dataset\Sea Bass
D:\st